In [41]:
import pandas as pd
import matplotlib.pyplot as plt
data_df = pd.read_csv("test.csv", index_col='no')
print("총 data size : ",data_df.shape[0] )

총 data size :  19537


In [42]:
data_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
no,,,,,,,,,,,,,,
29306,18,?,245274,Some-college,10,Never-married,?,Own-child,White,Male,0,0,16,United-States
29307,29,Private,83003,HS-grad,9,Married-civ-spouse,Other-service,Wife,White,Female,0,0,40,United-States
29308,45,Private,35136,Bachelors,13,Married-civ-spouse,Tech-support,Husband,Black,Male,0,0,40,United-States
29309,42,Self-emp-not-inc,64631,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States
29310,41,Private,195821,Doctorate,16,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,1902,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48838,45,Self-emp-not-inc,116789,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,60,United-States
48839,48,Private,185079,HS-grad,9,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,50,United-States
48840,63,Private,117473,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,4386,0,40,United-States


# education-num 처리

In [43]:
data_df[data_df['education-num'].isnull()].shape
data_df['education-num'].unique()

array([10,  9, 13, 16,  6, 11, 14,  4,  5,  7, 15, 12,  3,  8,  2,  1])

In [44]:
b = [0,3,6,9,11,13,40]
label = ['E','M','H','C','U','MA']
data_df['education-num'] =  pd.cut(data_df['education-num'], bins=b, labels=label)

# workclass 처리

In [45]:
wc_level_dic = {
    0: ['Never-worked','Without-pay'],
    1: ['Federal-gov','Self-emp-inc','State-gov'],
    2: ['?','Local-gov','Self-emp-not-inc'],
    3: ['Private']
}
def filter_workclass(X):
    for key, value in wc_level_dic.items():
        if str(X).strip() in value:
            return key
data_df['workclass'] = data_df['workclass'].apply(filter_workclass)

# occupation

In [46]:
occupation_level_dic = {0: [' Other-service', ' Priv-house-serv'], 1: [' Handlers-cleaners'], 2: [' Adm-clerical', ' Machine-op-inspct', ' Farming-fishing', ' ?'], 3: [], 4: [' Craft-repair', ' Transport-moving'], 5: [' Tech-support', ' Sales'], 6: [' Protective-serv', ' Armed-Forces'], 7: [], 8: [' Prof-specialty'], 9: [' Exec-managerial']}
def filter_country(X):
    for key, value in occupation_level_dic.items():
        if X in value:
            return key

data_df['occupation'] = data_df['occupation'].apply(filter_country)

# national-country

In [47]:
data_df['native-country'] = data_df['native-country'].apply(lambda x : x if x ==' United-States' else 'others')

# one-hot encoding

In [48]:
final_data = data_df.drop(columns='education')
final_data2 = pd.get_dummies(final_data, columns=['workclass','education-num','marital-status','occupation','relationship','race','sex','native-country'])

# model 불러오기

In [49]:
import xgboost as xgb
xgb_mobel = xgb.XGBClassifier()
xgb_mobel.load_model('model.json')

In [50]:
result = xgb_mobel.predict(final_data2)

# submission.csv 읽기 및 쓰기

In [51]:
submission_df = pd.read_csv('sample_submission.csv')

In [52]:
submission_df = submission_df.drop(columns='income')

In [53]:
pd.DataFrame(result, columns=['income'])

,income
0,0
1,0
2,1
3,1
4,1
...,...
19532,0
19533,0
19534,1
19535,0


In [54]:
new_submission_df = pd.concat([submission_df, pd.DataFrame(result, columns=['income'])], axis=1)

In [55]:
new_submission_df

,no,income
0,29306,0
1,29307,0
2,29308,1
3,29309,1
4,29310,1
...,...,...
19532,48838,0
19533,48839,0
19534,48840,1
19535,48841,0


In [56]:
new_submission_df.to_csv('submission.csv', index=False)